# Ensuring that the Mesh Poses saved can be utilized!

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import open3d as o3d
import os

%matplotlib inline

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


## Goals
- Mesh is captured 30 seconds w/ camera pose
- Find a way to reconstruct what meshes have been captured (full 3d reconstruct w/ MeshLab)
- Find correlation between mesh poses and camera poses
- Perform unprojection to test if it works

In [4]:
# PATH = '/home/andang/workspace/CV_Lab/Hololens-Project/datasets/john_place/raw-05-09-18-46/'
PATH = '/home/andang/workspace/Computational_Vision/HL2/Hololens-Project/datasets/john_place/vid1/'
PATH_MESH = PATH + 'mesh/'
# PATH_MESH = PATH + 'gt_mesh/'

In [5]:
mesh_hmap = {}

for path in os.listdir(PATH_MESH):
    split_names = (path.split('.txt')[0]).split('_')
    info_type = split_names[1]
    timestamp = split_names[2]
    index = split_names[3]
    data = np.loadtxt(PATH_MESH+path) #load up info

    if not (timestamp in mesh_hmap):
        mesh_hmap[timestamp] = {}

    if not index in mesh_hmap[timestamp]:
        mesh_hmap[timestamp][index] = {}


    mesh_hmap[timestamp][index][info_type] = data

In [8]:
#open3d rendering
open3d_meshes = []

for timestamp,j in mesh_hmap.items():
    for i,j in mesh_hmap[timestamp].items():
        vertex_positions = j['vertex']
        triangle_indices = j['triangles']
        pose = j['pose']
        vertex_position_scale = j['vertexscale']
        vertex_normals = j['normals']


        vertex_positions[:, 0:3] = vertex_positions[:, 0:3] * vertex_position_scale
        vertex_positions = (vertex_positions/vertex_positions[:,3:]) @ pose
        
        d = np.linalg.norm(vertex_normals, axis=1)
        vertex_normals[d > 0, :] = vertex_normals[d > 0, :] / d[d > 0, np.newaxis]
        vertex_normals = vertex_normals @ pose

        open3d_mesh = o3d.geometry.TriangleMesh()
        open3d_mesh.vertices = o3d.utility.Vector3dVector(vertex_positions[:,0:3])
        open3d_mesh.vertex_colors = o3d.utility.Vector3dVector(vertex_normals[:, 0:3])
        open3d_mesh.triangles = o3d.utility.Vector3iVector(triangle_indices)
        open3d_meshes.append(open3d_mesh)
    break

o3d.visualization.draw_geometries(open3d_meshes, mesh_show_back_face=True)

In [9]:
full_mesh = o3d.geometry.TriangleMesh()
for open3d_mesh in open3d_meshes:
    full_mesh += open3d_mesh

In [10]:
o3d.io.write_triangle_mesh('test_stuff.obj',full_mesh,write_ascii=True,write_triangle_uvs=True, write_vertex_colors=True, write_vertex_normals=True)

True

## SUCCESS! Open3D can open a 3d reconstruction mesh now